In [ ]:
import numpy as np
import tkinter as tk
from tkinter import ttk

#this function initializes the population with random solutions....
def initialize_population(pop_size, N):
    return np.random.randint(N, size=(pop_size, N))

#this func calculates the fitness of each solution in the population....
def calculate_fitness(population):
    fitness_values = []
    for x in population:
        penalty = 0
        N = len(x)
        #Looping through each queen's position....
        for i in range(N):
            r = x[i]
            #Checking for conflicts with other queens....
            for j in range(N):
                if i == j:
                    continue
                d = abs(i - j)
                if x[j] in [r, r-d, r+d]:
                    penalty += 1
        fitness_values.append(penalty)
    return -1 * np.array(fitness_values)  #Returning the negative penalty as fitness...

#selecting of individuals for reproduction....
def selection(population, fitness_values):
    probabilities = fitness_values.copy()  # corrected typo
    probabilities += abs(probabilities.min()) + 1
    probabilities = probabilities / probabilities.sum()
    N = len(population)
    indices = np.arange(N)
    #selecting individuals based on their fitness probabilities...
    selected_indices = np.random.choice(indices, size=N, p=probabilities)
    selected_population = population[selected_indices]
    return selected_population

#function fo crossover between two parents to produce offspring...
def crossover(parent1, parent2, pc):
    r = np.random.random()
    if r < pc:
        m = np.random.randint(1, len(parent1))
        child1 = np.concatenate([parent1[:m], parent2[m:]])
        child2 = np.concatenate([parent2[:m], parent1[m:]])
    else:
        child1 = parent1.copy()
        child2 = parent2.copy()
    return child1, child2


def mutation(individual, pm):
    r = np.random.random()
    if r < pm:
        m = np.random.randint(len(individual))
        individual[m] = np.random.randint(len(individual))
    return individual

#this function is to perform crossover and mutation on the selected individuals...
def crossover_mutation(selected_pop, pc, pm):
    N = len(selected_pop)
    new_pop = np.empty((N, len(selected_pop[0])), dtype=int)
    for i in range(0, N, 2):
        parent1 = selected_pop[i]
        parent2 = selected_pop[i+1]
        child1, child2 = crossover(parent1, parent2, pc)
        new_pop[i] = child1
        new_pop[i+1] = child2
    for i in range(N):
        mutation(new_pop[i], pm)
    return new_pop

#-----------------------------------------------------------------------------
# Main function......
def N_queens(pop_size, max_generations, N, pc=0.7, pm=0.01):
    # Initialization...
    population = initialize_population(pop_size, N)
    best_fitness_overall = None
    best_solution = None  # Initialize best_solution
    # Main loop for generations...
    for i_gen in range(max_generations):
        fitness_values = calculate_fitness(population)
        best_i = fitness_values.argmax()
        best_fitness = fitness_values[best_i]
        # Updating the best solution found so far...
        if best_fitness_overall is None or best_fitness > best_fitness_overall:
            best_fitness_overall = best_fitness
            best_solution = population[best_i].copy()  # Save a copy of the best solution
        print(f'\rgen={i_gen+1:06} -f={-best_fitness:03}', end='')
        # Check for optimal solution...
        if best_fitness == 0:
            print('\n found optimal solution')
            break
        # Selection, crossover and mutation...
        selected_pop = selection(population, fitness_values)
        population = crossover_mutation(selected_pop, pc, pm)
    print()
    # printing the best solution found.....
    print("The solution for", N, "queens (using genetic algorithm):")
    print(best_solution)
    return best_solution  # Return the best solution found



def print_board(solution):
    N = len(solution)
    board = np.zeros((N, N), dtype=int)
    for i in range(N):
        board[i, solution[i]] = 1
    for row in board:
        print("|", end="")
        for cell in row:
            if cell == 0:
                print("_|", end="")
            else:
                print("Q|", end="")
        print()

# Backtracking Algorithm....
#--------------------------------------------------------------------------

#the placeQ is used to check if we can place a queen in this position or not 
#b = chessboard // r = row // c = column // N = size of board(N*N)
def PlaceQ(b, r, c, N):

    # Check if there is a queen in the same column
    for i in range(r):
        if b[i][c] == 'Q':
            return False

    # Check up on the right side(fo2 el Q 3ala el right)
    for i, j in zip(range(r, -1, -1), range(c, N)):
        if b[i][j] == 'Q':
            return False

    # Check up on the left side(fo2 el Q 3ala el left)
    for i, j in zip(range(r, -1, -1), range(c, -1, -1)):
        if b[i][j] == 'Q':
            return False

    return True



def solve_NQueens(b, r, N):

    #Here it checks if the row number is equal the size of the chessboard(so it will not exced the size of the chessboard)
    if r == N:
        return True

    #Here it checks if the column number is in the range of the size of the chessboard(N)
    for c in range(N):
        if PlaceQ(b, r, c, N):
            b[r][c] = 'Q'
            if solve_NQueens(b, r + 1, N):
                return True
            b[r][c] = '-'

    return False


def solution(N):
    board = [['-'] * N for _ in range(N)]

    if not solve_NQueens(board, 0, N):
        print("No solution")
        return

    print("The solution for", N, "queens (using backtracking algorithm):")
    for r in board:
      #Join all items in a tuple into a string, using a sapce as a separator:
        print(" ".join(r))


class NQueensGUI:
    def __init__(self, master):
        self.master = master
        master.title("N-Queens Problem")

        # Create radio buttons for algorithm selection
        self.algorithm_var = tk.IntVar()
        self.algorithm_var.set(1)  # Default: Genetic Algorithm
        tk.Label(master, text="Select Algorithm:").grid(row=0, column=0, sticky="w")
        tk.Radiobutton(master, text="Genetic Algorithm", variable=self.algorithm_var, value=1).grid(row=0, column=1, sticky="w")
        tk.Radiobutton(master, text="Backtracking", variable=self.algorithm_var, value=2).grid(row=0, column=2, sticky="w")

        # Create text field for chessboard size
        tk.Label(master, text="Enter Chessboard Size:").grid(row=1, column=0, sticky="w")
        self.size_entry = tk.Entry(master)
        self.size_entry.grid(row=1, column=1)

        # Create button to solve the problem
        self.solve_button = tk.Button(master, text="Solve", command=self.solve_problem)
        self.solve_button.grid(row=1, column=2)

        # Create display area for solution
        self.solution_label = tk.Label(master, text="")
        self.solution_label.grid(row=2, column=0, columnspan=3)

        self.chessboard_canvas = tk.Canvas(master, width=400, height=400)
        self.chessboard_canvas.grid(row=3, column=0, columnspan=3)
        self.best_solution = None  # Initialize best_solution in the GUI

    def solve_problem(self):
        size = int(self.size_entry.get())
        algorithm = self.algorithm_var.get()

        if algorithm == 1:
            # Genetic Algorithm
            self.best_solution = N_queens(pop_size=100, max_generations=10000, N=size, pc=0.5, pm=0.05)
            if self.best_solution is not None:  # Check if solution is found
                self.display_solution(self.best_solution, size, "Genetic Algorithm")
            else:
                self.solution_label.config(text="No solution found.")
        else:
            # Backtracking Algorithm
            board = [['-'] * size for _ in range(size)]
            solve_NQueens(board, 0, size)
            self.display_chessboard(board)

    def display_solution(self, solution, size, algorithm_name):
        solution_text = f"Solution for {size}x{size} chessboard using {algorithm_name}:\n{solution}"
        self.solution_label.config(text=solution_text)
        if isinstance(solution[0], np.int64):
        # Convert the solution to a list of lists
            board = [['Q' if col == row else '-' for col in range(size)] for row in solution]
            self.display_chessboard(board)
        else:
            self.display_chessboard(solution)


    def display_chessboard(self, board):
        self.chessboard_canvas.delete("all")
        cell_size = 400 // len(board)
        for i, row in enumerate(board):
            for j, cell in enumerate(row):
                x1, y1 = j * cell_size, i * cell_size
                x2, y2 = x1 + cell_size, y1 + cell_size
                if isinstance(cell, str):
                    color = "black" if cell == "Q" else "white"
                else:
                    color = "black" if cell else "white"
                self.chessboard_canvas.create_rectangle(x1, y1, x2, y2, fill=color, outline="black")  # Add outline
                if color == "black":
                    text_x = x1 + cell_size // 2
                    text_y = y1 + cell_size // 2
                    self.chessboard_canvas.create_text(text_x, text_y, text="Q", fill="white")

    # Add outlines for the borders of the chessboard
        for i in range(len(board) + 1):
            self.chessboard_canvas.create_line(0, i * cell_size, 400, i * cell_size, fill="black")  # Horizontal lines
            self.chessboard_canvas.create_line(i * cell_size, 0, i * cell_size, 400, fill="black")  # Vertical lines


root = tk.Tk()
app = NQueensGUI(root)
root.mainloop()


# Entery...
if __name__ == "__main__":


        N = int(input("Please enter the size of the chessboard: "))  
        N_queens(pop_size=100, max_generations=10000, N=N, pc=0.5, pm=0.05)
        print('\n')
        
        try:
            solution(N)
        except ValueError:
            print("Please enter the size of the board (must be an integer (4,8 or 16)).")




gen=001209 -f=000
 found optimal solution

The solution for 16 queens (using genetic algorithm):
[10 13  7  4  2 12  6  1 14 11  3  8 15  5  9  0]
gen=004319 -f=000
 found optimal solution

The solution for 12 queens (using genetic algorithm):
[ 7 10  2  5  1  9  4  8 11  0  3  6]
